In [1]:
import numpy as np
import pandas as pd
import gc
import time
import category_encoders as ce
from contextlib import contextmanager
import lightgbm as lgb
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from scipy.cluster.vq import kmeans2, whiten
from sklearn.preprocessing import Imputer
from sklearn.decomposition import truncated_svd
import category_encoders as ce
from catboost import CatBoostClassifier
from sklearn import preprocessing
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print('[{' + name + '}] done in {' + str(round(time.time() - t0, 3)) + '} s')

num_rows = None
EPS = 1e-100

/home/kain/Workstation/PyEnv/lib/python3.5/site-packages/sklearn/externals/joblib/_multiprocessing_helpers.py:28: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))
/home/kain/Workstation/PyEnv/lib/python3.5/site-packages/joblib/_multiprocessing_helpers.py:28: UserWarning: [Errno 13] Permission denied.  joblib will operate in serial mode
  warnings.warn('%s.  joblib will operate in serial mode' % (e,))


In [30]:
test_file_path = "Level_1_stack/test_ann_suresh_features2_neptune_1.csv"
validation_file_path = 'Level_1_stack/validation_ann_suresh_features2_neptune_1.csv'
num_folds = 5

In [3]:
test = pd.read_csv('/media/limbo/Home-Credit/data/application_test.csv', nrows=None)

In [4]:

train = pd.read_csv('/media/limbo/Home-Credit/data/application_train.csv.zip', nrows= num_rows)
n_train = train.shape[0]

In [19]:
df = pd.read_csv('../data/selected_features2.csv', header=0, index_col=None)

In [20]:
df.head()

,NAME_CONTRACT_TYPE_SURESH,CODE_GENDER_SURESH,FLAG_OWN_CAR_SURESH,AMT_INCOME_TOTAL_SURESH,AMT_ANNUITY_SURESH,AMT_GOODS_PRICE_SURESH,NAME_INCOME_TYPE_SURESH,NAME_EDUCATION_TYPE_SURESH,NAME_HOUSING_TYPE_SURESH,REGION_POPULATION_RELATIVE_SURESH,...,FLAG_DOC_SUM_SURESH,DEF_30_60_SOCIAL_SUM_SURESH,OBS_30_60_SOCIAL_SUM_SURESH,TOTAL_CREDIT_CHECKS_YEAR_SURESH,PER_CREDIT_CHECKS_MON_SURESH,TOTAL_CREDIT_CHECKS_MON_SURESH,TOTAL_CREDIT_CHECKS_QRT_SURESH,PER_CREDIT_CHECKS_MON_3MON_SURESH,LIVE_IND_SUM_SURESH,NEW_INC_BY_ORG_SURESH
0,1,2,1,202500.0,24700.5,351000.0,5,4,2,0.018801,...,1,4,4,768216,0.110012,84513,106619,0.792664,NaN,157500.0
1,1,1,1,270000.0,35698.5,1129500.0,4,2,2,0.003541,...,1,0,2,768216,0.110012,84513,106619,0.792664,38.244017,135000.0
2,2,2,2,67500.0,6750.0,135000.0,5,4,2,0.010032,...,0,0,0,768216,0.110012,84513,106619,0.792664,NaN,135000.0
3,1,1,1,135000.0,29686.5,297000.0,5,4,2,0.008019,...,1,0,4,768216,0.110012,84513,106619,0.792664,51.000000,157500.0
4,1,2,1,121500.0,21865.5,513000.0,5,4,2,0.028663,...,0,0,0,768216,0.110012,84513,106619,0.792664,48.147723,157500.0


In [21]:
df.shape

(356255, 547)

In [22]:
neptune = pd.read_csv('../data/neptue_features_0.csv', header=0, index_col=0)

In [23]:
gc.collect()

7

In [24]:
neptune.head()

,index,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,...,POPULAR_AMT_GOODS_PRICE_SURESH,INSTALLMENT_PAID_LATE_DAYS_LAST_LOAN_MEAN_SURESH,MEAN_PAYMENT_CC_SURESH,DAYS_EMPLOYED_DIFF_DAYS_BIRTH_SURESH,TREND_COEF_PAID_LATE_SURESH,AMT_INCOME_TOTAL_12_AMT_ANNUITY_SURESH,number_of_installments_paid_late_03,mean_number_of_days_paid_late_03,max_number_of_days_paid_late_03,mean_amont_paid_late_03
0,0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,...,0,-20.421053,NaN,8824.0,0.157343,-7825.5,NaN,NaN,NaN,NaN
1,1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,...,0,-4.428571,NaN,15577.0,NaN,-13198.5,NaN,NaN,NaN,NaN
2,2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,...,0,-7.666667,NaN,18821.0,NaN,-1125.0,NaN,NaN,NaN,NaN
3,3,29686.5,312682.5,297000.0,135000.0,NaN,NaN,NaN,NaN,NaN,...,0,-4.500000,0.0,15966.0,1.181081,-18436.5,NaN,NaN,NaN,NaN
4,4,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,0.0,0.0,...,0,-2.250000,NaN,16894.0,-0.504714,-11740.5,NaN,NaN,NaN,NaN


In [25]:
uniuqes = [f for f in neptune.columns if f not in df.columns]

In [26]:
df = pd.concat([df, neptune[uniuqes]], axis=1)

#df = neptune

In [27]:
df.shape

(356255, 884)

In [28]:
import logging
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
from keras.callbacks import EarlyStopping, ModelCheckpoint
import keras as ks
from keras import backend as K
import gc
from contextlib import contextmanager
import tensorflow as tf

class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: {:d} - score: {:.6f}".format(epoch+1, score))

In [29]:
df = df * 1

In [ ]:
encoding = 'ohe'
gc.collect()
train_df = df.iloc[0:n_train]
test_df = df.iloc[n_train:]

print("Starting ANN. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
gc.collect()
# Cross validation model
folds = KFold(n_splits=num_folds, shuffle=True, random_state=1001)
# Create arrays and dataframes to store results
oof_preds = np.zeros(train_df.shape[0])
sub_preds = np.zeros(test_df.shape[0])
feature_importance_df = pd.DataFrame()
feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV','index']]

#feats = [col for col in feats_0 if df[col].dtype == 'object']


print(train_df[feats].shape)
for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train['TARGET'])):
        
        categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
        
        if encoding == 'ohe':
            
            enc = ce.OneHotEncoder(impute_missing=True, cols=categorical_columns).fit(train_df[feats].iloc[train_idx],
                                                                                       train['TARGET'].iloc[train_idx])
            x_train = enc.transform(train_df[feats].iloc[train_idx]).replace([-np.inf, np.inf]).fillna(-999)
            x_valid = enc.transform(train_df[feats].iloc[valid_idx]).replace([-np.inf, np.inf]).fillna(-999)
            x_test = enc.transform(test_df[feats]).replace([-np.inf, np.inf]).fillna(-999)
            gc.collect()
            scaler = preprocessing.RobustScaler(quantile_range=(5.0, 95.0), with_scaling=True, with_centering=True)
            scaler.fit(x_train)
            x_train = scaler.transform(x_train)
            x_valid = scaler.transform(x_valid)
            x_test = scaler.transform(x_test)
            
            print(x_train.shape, x_valid.shape, x_test.shape)
        
        file_path = "best_model.hdf5"
        check_point = ModelCheckpoint(file_path, monitor="val_loss", verbose=1,
                              save_best_only=True, mode="min")
        ra_val = RocAucEvaluation(validation_data=(x_valid, train['TARGET'].iloc[valid_idx].values), interval=1)
        early_stop = EarlyStopping(monitor="val_loss", mode = "min", patience=2)
        gc.collect()
        
        config = tf.ConfigProto(
        intra_op_parallelism_threads=6, use_per_session_threads=6, inter_op_parallelism_threads=6)
        with tf.Session(graph=tf.Graph(), config=config) as sess, timer('fit_predict'):
                    ks.backend.set_session(sess)
                    model_in = ks.Input(shape=(x_train.shape[1],), dtype='float32', sparse=False)
                    out = ks.layers.Dense(2 ** 10,  activation='sigmoid', kernel_initializer=
                      ks.initializers.RandomNormal(mean=0.00, stddev=0.05, seed=666))(model_in)
                    out = ks.layers.Dropout(0.5)(out)
                    out =  ks.layers.Dense(2 ** 9, activation='sigmoid', kernel_initializer=
                      ks.initializers.RandomNormal(mean=0.00, stddev=0.05, seed=666))(out)
                    out = ks.layers.Dropout(0.3)(out)
                    out =  ks.layers.Dense(2 ** 8, activation='relu', kernel_initializer=
                      ks.initializers.RandomNormal(mean=0.00, stddev=0.05, seed=666))(out)
                    out = ks.layers.Dropout(0.2)(out)
                    out = ks.layers.Dense(1, activation='sigmoid', kernel_initializer=
                      ks.initializers.RandomNormal(mean=0.00, stddev=0.05, seed=666))(out)
                    model = ks.models.Model(model_in, out)
                    model.compile(loss='binary_crossentropy',
                                  optimizer=ks.optimizers.Adam(lr=1e-3), metrics=['binary_crossentropy'])
                    batch_size = 2 ** 10
                    for i in range(3):
                        with timer('pass ' +  str(i + 1)):
                            model.fit(x=x_train, y=train['TARGET'].iloc[train_idx].values, batch_size=batch_size+(batch_size*(2*i)), epochs=5, 
                                validation_data=(x_valid, train['TARGET'].iloc[valid_idx].values), callbacks=[ra_val, check_point, early_stop], shuffle=True)
                    y_pred = model.predict(x_valid).reshape(-1, 1)
                    print(roc_auc_score(y_true=train['TARGET'].iloc[valid_idx].values, y_score=y_pred))
                    gc.collect()   
                    oof_preds[valid_idx] = model.predict(x_valid)[:, 0]
                    sub_preds += model.predict(x_test)[:, 0] / folds.n_splits
                    gc.collect()

print('Full AUC score %.6f' % roc_auc_score(train['TARGET'], oof_preds))
             


Starting ANN. Train shape: (307511, 884), test shape: (48744, 884)
(307511, 881)
(246008, 1036) (61503, 1036) (48744, 1036)
Train on 246008 samples, validate on 61503 samples
Epoch 1/5
246008/246008 [==============================] - 36s 146us/step - loss: 0.2689 - binary_crossentropy: 0.2689 - val_loss: 0.2551 - val_binary_crossentropy: 0.2551

 ROC-AUC - epoch: 1 - score: 0.748999

Epoch 00001: val_loss improved from inf to 0.25515, saving model to best_model.hdf5
Epoch 2/5
246008/246008 [==============================] - 35s 142us/step - loss: 0.2506 - binary_crossentropy: 0.2506 - val_loss: 0.2541 - val_binary_crossentropy: 0.2541

 ROC-AUC - epoch: 2 - score: 0.753844

Epoch 00002: val_loss improved from 0.25515 to 0.25405, saving model to best_model.hdf5
Epoch 3/5
246008/246008 [==============================] - 35s 142us/step - loss: 0.2485 - binary_crossentropy: 0.2485 - val_loss: 0.2520 - val_binary_crossentropy: 0.2520

 ROC-AUC - epoch: 3 - score: 0.758320

Epoch 00003: val_

246009/246009 [==============================] - 33s 134us/step - loss: 0.2434 - binary_crossentropy: 0.2434 - val_loss: 0.2442 - val_binary_crossentropy: 0.2442

 ROC-AUC - epoch: 1 - score: 0.766924

Epoch 00001: val_loss improved from 0.24433 to 0.24416, saving model to best_model.hdf5
Epoch 2/5
246009/246009 [==============================] - 33s 134us/step - loss: 0.2429 - binary_crossentropy: 0.2429 - val_loss: 0.2441 - val_binary_crossentropy: 0.2441

 ROC-AUC - epoch: 2 - score: 0.766661

Epoch 00002: val_loss improved from 0.24416 to 0.24410, saving model to best_model.hdf5
Epoch 3/5
246009/246009 [==============================] - 33s 134us/step - loss: 0.2428 - binary_crossentropy: 0.2428 - val_loss: 0.2443 - val_binary_crossentropy: 0.2443

 ROC-AUC - epoch: 3 - score: 0.767578

Epoch 00003: val_loss did not improve from 0.24410
Epoch 4/5
246009/246009 [==============================] - 33s 134us/step - loss: 0.2425 - binary_crossentropy: 0.2425 - val_loss: 0.2441 - val_bin

In [ ]:
print('Full AUC score %.6f' % roc_auc_score(train['TARGET'], oof_preds))
        
sub_df = test[['SK_ID_CURR']].copy()
sub_df['TARGET'] = sub_preds
sub_df[['SK_ID_CURR', 'TARGET']].to_csv(test_file_path, index= False)

val_df = train[['SK_ID_CURR', 'TARGET']].copy()
val_df['TARGET'] = oof_preds
val_df[['SK_ID_CURR', 'TARGET']].to_csv(validation_file_path, index= False)        
            
            
        
        
    

